# Energy/Source API

This document is a mockup of an experimental package for calculating source parameters in a plethora of ways. 

## EventSource

EventSource is a high-level object which provides a simple API for calculating magnitudes and other basic source parameters.

In [ ]:
from energy import EventSource

# get basic energy, fc, moment, etc.
cat = get_catalog()  # just a catalog, nothing fancy
waveform_client = obsplus.WaveBank('directory_of_waveforms')
inv = obspy.read_inventory('inventory_path.xml')

event = cat[0]

First an example of how to calculate basic magnitudes attach them to an event.

By default this will calculate MW, ME, and ML, and mark MW as preferred.

In [ ]:
source = EventSource(event, waveform_client, inv)
new_event = source.update_event(event)

A simple series with various source parameters and error estimates is also possible. This includes station aggregated moment, local magnitude, moment magnitude, corner frequencies, etc.

In [ ]:
source.get_source_summary()

The above functionality will satisfy the needs of most users.

However, to get more details about what is occurring under the hood, or to conduct more in-depth source parameter studies, information can be returned about the source parameters and their values used to extimate them from each station. To get a dataframe with multi-index columns describing all the supported methods (usually identified by author name of reference paper), then by seed_id as an index: 

In [ ]:
df = source.get_detailed_source_df()

## Advanced Customization

Moreover, there many tunable parameters that can be defined by a subclass of `SourceController`. This allows the user to do a few things:

    1) Define custom pre-processing steps like adjusting for geometric spreading or intrinsic attenuation.
    2) Define a simple earth model for determining Q, velocity, or density values given a source location and receiver location
    3) Custom hooks for dumping outputs of the analysis at various stages

In [ ]:
from energy import SourceController

# First define custom behavior
class MySourceController(SourceController):
    
    def get_p_velocity(self):
        """ A custom function for returning a P wave velocity """
        if self.origin.depth > 1000:  # depth greater than 1000 m
            return 6_000
        else:
            return 4_500
        
    def correct_site_effects(self, spectrum):
        """ A custom adustment for site effects. """
        if self.station_name == 'TMU':
            return spectrum * 2
    
    def correct_geometric_spreading(self, spectrum):
        """ Correct for geometric spreading """
        factor = look_up_some_factor(self.event, self.station)
        return spectrum * factor
    

In [ ]:
# Now calculate source paramters using behavior defined in MySourceController
with MySourceController:
    source = EventSource(event, waveform_client, inv)

# Spectra
A spectral representation of seismic data is needed to determine source parameters. These handy little classes forms the main building blocks for source parameter determination so a brief demo is in order.

The two spectral classes are `EventSpectra` and `EventSpectrum`. They are a close parity to obspys' `Stream` and `Trace`, respectively. 

In [ ]:
import obspy
from energy import EventSpectra, EventSpectrum

# EventSpectrum is created from a trace with the response removed

# get stream and remove response, ensure velocity is motion_type
st = obspy.read()
inv = obspy.read_events()
st.remove_response(inv, output='VEL')

spectrum = EventSpectrum(st[0], 'velocity')

# or Spectra can be created from a stream
spectra = EventSpectra(st, 'velocity')

In [ ]:
# When a spectrum is calculated the displacement, velocity, and acceleration for the input
# is determined. A dataframe with indicies as frequencies and columns of Displacement, Velocity,
# Acceleration is at the core of each spectrum. 